
## Segmenting and Clustering Neighbourhood 
### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

<h4> 1.Cleaning, storing and finding latitude and longitude(code from previous two notebook)

In [2]:
## IBM on Coursera Capstone Project
### This notebook mainly use for Data Science Capstone project

import numpy as np
import pandas as pd

print("Hello Capstone Project Course!")

def extract_column(word):

    clip=word[word.find("<th>"):] #remove reduncdance 
    list=[]
    
    for i in range(word.count("<th>")) :
        temp=clip[clip.find("<th>")+4:clip.find("</th>")]
        forremove=clip[clip.find("<th>"):clip.find("</th>")+5]
        list.append(temp)
        clip=clip.replace(forremove,"")
         
    return list


def extract_row(word):
    
  
    clip=word[word.find("<tr>"):] #remove reduncdance 
    list=[]
    
    for i in range(word.count("<tr>")) :
        temp=clip[clip.find("<tr>")+4:clip.find("</tr>")]
        forremove=clip[clip.find("<tr>"):clip.find("</tr>")+5]
        list.append(temp)
        clip=clip.replace(forremove,"")
        
    return list

def extract_td(word):
    
    #prep
    clip=word[word.find("<td>"):] #remove reduncdance 
    list=[]

    
    for i in range(word.count("<td>")) :
        
        temp=clip[clip.find("<td>")+4:clip.find("</td>")]
        
        if temp.find("title")!=-1:
            temp=temp[temp.find('>')+1:temp.find('</a>')]
            
        forremove=clip[clip.find("<td>"):clip.find("</td>")+5]
        list.append(temp)
        clip=clip.replace(forremove,"")
        
    return list

import urllib.request as openner #url library

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Request and Parsing html file
f=openner.urlopen(url)
page=f.read().decode("utf-8")
f.close()


#Extract table component
clip = page[page.find("<table"):page.find("</table>")]
columns = clip[clip.find("<tr>"):clip.find("</tr>")]
column=extract_column(columns)
clip_row = clip.replace(columns,"")

#Extract Row Component
listoftr=extract_row(clip_row)

#Extract Table Data Component
listoftd=[]
for content in listoftr:
    listoftd.append(extract_td(content))

df=pd.DataFrame(listoftd,columns=column)
df



#Drop row that contain 'Not assigned' in 'Borough' 
drop_df=df.loc[df.Borough!='Not assigned'].reset_index().drop('index',axis=1)

#clean \n
drop_df.columns=['Postcode', 'Borough', 'Neighbourhood']

#clean \n in data and check for 'Not assigned' in Neighbourhood column
for i,neigh in enumerate(drop_df['Neighbourhood']):
    if neigh.find('\n')!=-1:
        drop_df.loc[i,'Neighbourhood']=neigh[:-1]
        
    if drop_df.loc[i,'Neighbourhood'] == 'Not assigned':
        drop_df.loc[i,'Neighbourhood']=drop_df.loc[i,'Borough']
    
drop_df


# Merge Function 
#df is Dataframe , index column that use for Grouping

def merge(df,index):
    
    newdf=pd.DataFrame(columns=drop_df.columns.values)
    previous=''
    j=0

    for i,postcode in enumerate(drop_df[index]):
        
        if postcode!=previous :
            newdf.loc[j]=drop_df.loc[i]
            newdf.iloc[j,2]=drop_df.iloc[i,2]
            j+=1
            
        else:
            index=j-1
            newdf.iloc[index,2]=newdf.iloc[index,2]+','+drop_df.iloc[i,2]
        previous=postcode
        
    return newdf

test=merge(drop_df,'Postcode')
test

!pip install geocoder

#Cant fetch the data via geocoder
'''import geocoder # import geocoder
# loop until you get the coordinates
for i,postcode in enumerate(drop_df['Postcode']):
    lat_lng_coords = None
    j=0
    while(j < 100):
      g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
      lat_lng_coords = g.latlng
      j+=1
    print(postcode,' ',lat_lng_coords)
    #drop_df.loc[i,'latitude'] = lat_lng_coords[0]
    #drop_df.loc[i,'longitude']  = lat_lng_coords[1]
'''    

geo=pd.read_csv("https://cocl.us/Geospatial_data")

for i,postcode in enumerate(test['Postcode']):
    
    for j,postcheck in enumerate(geo['Postal Code']):
        if postcode==postcheck:
            test.loc[i,'Latitude'] = geo.loc[j,'Latitude']
            test.loc[i,'Longitude']  = geo.loc[j,'Longitude']
            break
test    

Hello Capstone Project Course!
    100% |████████████████████████████████| 102kB 7.4MB/s ta 0:00:01
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: urll

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


<h4>2. Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  50.36 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.49 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.61 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  45.67 MB/s
Libraries imported.


<h4> 3.Define Foursqure Credential

In [4]:
CLIENT_ID = '3BSP2NJQZKJGGXN0COE5P40VD2MMHXFWML50L3VEBPQRJI4S' # your Foursquare ID
CLIENT_SECRET = 'DK2HY04ZUFHSY2KFYAOJVY5IK1TQYC0ML5CT20EICLGA50VZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3BSP2NJQZKJGGXN0COE5P40VD2MMHXFWML50L3VEBPQRJI4S
CLIENT_SECRET:DK2HY04ZUFHSY2KFYAOJVY5IK1TQYC0ML5CT20EICLGA50VZ


<h4> 4.Define getNearbyVenues function to fetch the nearby venue for all neighborhood and return a Dataframe of result

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=test['Neighbourhood'],
                                   latitudes=test['Latitude'],
                                   longitudes=test['Longitude']
                                  )
toronto_venues.head()

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


<h4> 5.Merging test with toronto_venues  (using join method)

In [35]:
toronto_venues.set_index('Neighborhood',inplace=True)
test.set_index('Neighbourhood',inplace=True)

test2=test.join(toronto_venues,lsuffix='Neighbourhood',rsuffix='Neighborhood')
test2.reset_index(inplace=True)
test2.rename(columns={'index':'Neighborhood'},inplace=True)
test2.set_index('Postcode',inplace=True)
test2.reset_index(inplace=True)
test2.head(20)

,Postcode,Neighborhood,Borough,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650609,-79.386280,Concert Hall
1,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
3,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
4,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel
5,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Bosk at Shangri-La,43.649023,-79.385826,Asian Restaurant
6,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
7,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,M Square Coffee Co,43.651218,-79.383555,Coffee Shop
8,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Soho House Toronto,43.648734,-79.386541,Speakeasy
9,M5H,"Adelaide,King,Richmond",Downtown Toronto,43.650571,-79.384568,43.650571,-79.384568,Momofuku Noodle Bar,43.649366,-79.386217,Noodle House


In [29]:
test2.groupby('Neighborhood').count()

,Postcode,Borough,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11,11,11,11,11
"Alderwood,Long Branch",10,10,10,10,10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",27,27,27,27,27,27,27,27,27,27
Berczy Park,57,57,57,57,57,57,57,57,57,57


In [30]:
print('There are {} uniques categories.'.format(len(test2['Venue Category'].unique())))

There are 278 uniques categories.


<h4> 6. Sorting most venue category in each neighborhood(use Postcode to find corodinate)

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(test2[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] =b test2['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.reset_index(inplace=True)
toronto_onehot.set_index('Neighborhood',inplace=True)
toronto_onehot.reset_index(inplace=True)

toronto_onehot.drop(columns='index',inplace=True)
toronto_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0,0,0,0,0,0

In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.

In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.04
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
                      venue  freq
0              Skating Rink  0.25
1                    Lounge  0.25
2            Breakfast Spot  0.25
3            Sandwich Place  0.25
4  Mediterranean Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0                 Gym  0.33
1          Playground  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1          Pizza Place  0.09
2  Fried Chicken Joint  0.09
3       Sandwich Place  0.09
4       Discount Store  0.09


----Alderwood,Long Branch----
            venue  f

                        venue  freq
0            Basketball Court  0.25
1                        Park  0.25
2                      Bakery  0.25
3  Construction & Landscaping  0.25
4   Middle Eastern Restaurant  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0  Convenience Store  0.17
1        Coffee Shop  0.17
2        Bus Station  0.17
3     Discount Store  0.17
4   Department Store  0.17


----East Toronto----
                       venue  freq
0                       Park  0.50
1                Coffee Shop  0.25
2          Convenience Store  0.25
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Emery,Humberlea----
                    venue  freq
0  Furniture / Home Store   0.5
1          Baseball Field   0.5
2       Martial Arts Dojo   0.0
3          Massage Studio   0.0
4          Medical Center   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restauran

                venue  freq
0              Garden   0.5
1        Home Service   0.5
2         Yoga Studio   0.0
3  Miscellaneous Shop   0.0
4               Motel   0.0


----Rouge,Malvern----
                  venue  freq
0  Fast Food Restaurant   0.5
1            Print Shop   0.5
2           Yoga Studio   0.0
3    Mexican Restaurant   0.0
4   Monument / Landmark   0.0


----Runnymede,Swansea----
                venue  freq
0         Coffee Shop  0.10
1         Pizza Place  0.08
2                Café  0.08
3    Sushi Restaurant  0.05
4  Italian Restaurant  0.05


----Ryerson,Garden District----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.07
2                       Café  0.04
3             Cosmetics Shop  0.03
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                venue  freq
0          Playground   0.5
1      Cosmetics Shop   0.5
2         Yoga Studio   0.0
3  Miscellaneous Shop   0.0
4               Mo

Define 'return_most_common_venues' (use for return the most frequent venue into list of venue category name.)

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Restaurant,Burger Joint,Bar,Bakery,Hotel
1,Agincourt,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Drugstore,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dance Studio,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Gym,Park,Playground,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Discount Store,Beer Store,Fast Food Restaurant,Japanese Restaurant,Coffee Shop
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Pool,Coffee Shop,Skating Rink,Sandwich Place,Pub,Athletics & Sports,Gym,College Stadium


<h4>7. Clustering using kmean

In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

merging sorted venue catagory list and test dataframe

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = test

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'))

toronto_merged.head()

,Postcode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighbourhood,,,,,,,,,,,,,,,
Parkwoods,M3A,North York,43.753259,-79.329656,1,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
Victoria Village,M4A,North York,43.725882,-79.315572,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Cupcake Shop
"Harbourfront,Regent Park",M5A,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,Restaurant
"Lawrence Heights,Lawrence Manor",M6A,North York,43.718518,-79.464763,0,Clothing Store,Women's Store,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant,Airport Food Court
Queen's Park,M7A,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Japanese Restaurant,Gym,Diner,Yoga Studio,Burger Joint,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Seafood Restaurant


<h4>8. Create folium map represent each cluster with different color

In [77]:
# create map
map_clusters = folium.Map(location=[toronto_merged.Latitude.mean(), toronto_merged.Longitude.mean()], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + " ,Cluster : " + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters